In [1]:
## Get dependencies ##

import numpy as np
import string
import math
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
sys.path.append('..')
from GIR import *
import scipy as sp
import pickle
import time
import scipy as sp
from scipy import signal
from scipy.io.idl import readsav
import os
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import glob
import requests
import ftplib
import io
import cmocean
from bs4 import BeautifulSoup
import urllib.request
from io import StringIO, BytesIO
from zipfile import ZipFile
import tqdm
import seaborn as sn
import re

## get SSP emissions
from tools.RCMIP import *

header = {
  "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.75 Safari/537.36",
  "X-Requested-With": "XMLHttpRequest"
}
    
## Matplotlib rcparams setup:

matplotlib.rcParams['font.family']='Helvetica'
matplotlib.rcParams['font.size']=11
# matplotlib.rcParams['font.weight']=400

matplotlib.rcParams['image.cmap']='cmo.ice'

matplotlib.rcParams['axes.prop_cycle']=matplotlib.cycler('color',['011936','FF7D00','225560','BFACAA','D72638','788C9B','A33F00','7CAAB0','685655','EB767C'])
matplotlib.rcParams['axes.formatter.limits']=-3,3
# matplotlib.rcParams['axes.labelweight']=300

matplotlib.rcParams['legend.frameon']=False

matplotlib.rcParams['boxplot.whiskers']=(5,95)
matplotlib.rcParams['boxplot.showfliers']=False
matplotlib.rcParams['boxplot.showfliers']=False
matplotlib.rcParams['boxplot.medianprops.color']='black'

matplotlib.rcParams['errorbar.capsize']=5

matplotlib.rcParams['hist.bins']='auto'

plt.rcParams['pdf.fonttype'] = 42

%matplotlib inline

%load_ext line_profiler
%load_ext memory_profiler

/home/leachl/miniconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1762: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)
../tools/RCMIP.py:61: PerformanceWarning: indexing past lexsort depth may impact performance.
  GIR_to_RCMIP_map.loc[RCMIP_to_GIR_map_concs.values(),'RCMIP_concs_unit'] = RCMIP_concs.loc[('World','ssp245')].reindex(RCMIP_to_GIR_map_concs.keys()).loc[:,'Unit'].values#.loc[('World','ssp245',RCMIP_to_GIR_map_concs.keys()),'Unit'].values


In [2]:
## import concentrations:
CMIP6_conc_ftp = ftplib.FTP('data.iac.ethz.ch','anonymous')
CMIP6_conc_ftp.cwd('CMIP6/input4MIPs/UoM/GHGConc/CMIP/yr/atmos/UoM-CMIP-1-1-0/GHGConc/gr3-GMNHSH/v20160701')
CMIP6_ftp_list = [x for x in CMIP6_conc_ftp.nlst() if x[-3:]=='csv']
WMGHG_concs = pd.DataFrame(dict(zip(['_'.join(x.split('_')[3:-8]) for x in CMIP6_ftp_list],[pd.read_csv('ftp://data.iac.ethz.ch/CMIP6/input4MIPs/UoM/GHGConc/CMIP/yr/atmos/UoM-CMIP-1-1-0/GHGConc/gr3-GMNHSH/v20160701/'+x,usecols=[0,1],index_col=0).iloc[:,0] for x in CMIP6_ftp_list])))
WMGHG_concs = WMGHG_concs[[x for x in WMGHG_concs.columns if x[-2:]!='eq']] # remove "equivalent" concentrations

WMGHG_concs['halon1202'] = 0
WMGHG_concs.loc[1765:2014,'halon1202'] = pd.read_csv('http://www.pik-potsdam.de/~mmalte/rcps/data/RCP45_MIDYEAR_CONCENTRATIONS.DAT',skiprows=38,delim_whitespace=True,index_col=0)['HALON1202'].loc[1765:2014].values

NOAA_mole_fractions_2020 = pd.read_csv('https://www.esrl.noaa.gov/gmd/aggi/NOAA_MoleFractions_2020.csv',skiprows=2,index_col=0,skipfooter=5).iloc[1:].replace('nd',np.nan).apply(pd.to_numeric)

scale_factor = (WMGHG_concs.loc[2014,'carbon_dioxide'] / NOAA_mole_fractions_2020.loc[2014:2015,'CO2']).values

CO2_concs = pd.Series(np.append(WMGHG_concs.loc[1750:,'carbon_dioxide'].values.flatten(),NOAA_mole_fractions_2020.loc[2015:,'CO2']*scale_factor),index=np.arange(1750,2020),name='CO2')

/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  # This is added back by InteractiveShellApp.init_path()


In [3]:
for i,gas in enumerate(WMGHG_concs.columns):
    if gas in ['nitrous_oxide','carbon_dioxide','methane']:
        continue
    else:
        WMGHG_concs.loc[:,gas]/=1000

In [4]:
## get aerosol emissions for accurate historical temp reproduction

RCMIP_aer_emms = RCMIP_to_GIR_input_emms('historical').dropna().loc[:,[x for x in RCMIP_to_GIR_input_emms('historical').dropna().columns if x not in WMGHG_concs.columns]]

baseline_scenario = pd.concat([pd.concat([WMGHG_concs.loc[1750:2014],RCMIP_aer_emms-RCMIP_aer_emms.loc[1750]],axis=1)],axis=1,keys=['baseline'])

baseline_forc = pd.concat([get_RCMIP_forc('historical')],axis=1,keys=['baseline']).dropna().reindex(np.arange(1750,2516))
baseline_forc.loc[2015:2515] = baseline_forc.loc[2014].values[0]

baseline_scenario = baseline_scenario.reindex(np.arange(1750,2516))

for gas in baseline_scenario.columns.levels[1]:
    baseline_scenario.loc[2015:2515,('baseline',gas)] = baseline_scenario.loc[2014,('baseline',gas)]

In [5]:
baseline_emms = run_GIR(concentrations_in=baseline_scenario,forcing_in=baseline_forc,aer_concs_in=['bc', 'co', 'nh3', 'nmvoc', 'nox', 'nox_avi', 'oc', 'so2'])['Emissions'].droplevel(1,axis=1)

Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch2cl2', 'ch3ccl3', 'ch3ccl3|strat_o3', 'chcl3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc23', 'hfc236fa', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nf3', 

100%|██████████| 765/765 [00:00<00:00, 6687.68 timestep/s]


In [6]:
baseline_emms.loc[:2014,('baseline',RCMIP_aer_emms.columns)] = RCMIP_aer_emms.values
for aer in RCMIP_aer_emms.columns:
    baseline_emms.loc[2015:2515,('baseline',aer)] = RCMIP_aer_emms.loc[2014,aer]

In [7]:
baseline_run = run_GIR(emissions_in=baseline_emms,forcing_in=baseline_forc)

Integrating 1 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch2cl2', 'ch3ccl3', 'ch3ccl3|strat_o3', 'chcl3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc23', 'hfc236fa', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nf3', 

100%|██████████| 765/765 [00:00<00:00, 4742.30 timestep/s]


In [8]:
gas_mass_conversion_factors = pd.Series(index=baseline_emms.columns.levels[1])
gas_mass_conversion_factors.loc[:] = 1
gas_mass_conversion_factors.loc['carbon_dioxide'] = (1/1000)/(44.01/12.01)
gas_mass_conversion_factors.loc['nitrous_oxide'] = 28/44

/home/leachl/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [9]:
perturbed_runs = [baseline_emms.loc[:,'baseline']]
run_keys = ['baseline']

for gas in baseline_emms.columns.levels[1]:
    pert_emms = baseline_emms.loc[:,'baseline'].copy()
    pert_emms.loc[2015,gas] += gas_mass_conversion_factors.loc[gas]/1e6
    run_keys += [gas]
    perturbed_runs += [pert_emms]
    
experiment_emms = pd.concat(perturbed_runs,axis=1,keys=run_keys)

In [10]:
experiment_forc = pd.concat([baseline_forc['baseline']]*len(run_keys),axis=1,keys=run_keys)

In [11]:
experiment_run = run_GIR(emissions_in=experiment_emms,forcing_in=experiment_forc)

Integrating 53 scenarios, 1 gas cycle parameter sets, 1 thermal response parameter sets, over ['bc', 'bc|aci', 'bc|bc_on_snow', 'c2f6', 'c3f8', 'c4f10', 'c5f12', 'c6f14', 'c7f16', 'c8f18', 'c_c4f8', 'carbon_dioxide', 'carbon_tetrachloride', 'carbon_tetrachloride|strat_o3', 'cf4', 'cfc11', 'cfc113', 'cfc113|strat_o3', 'cfc114', 'cfc114|strat_o3', 'cfc115', 'cfc115|strat_o3', 'cfc11|strat_o3', 'cfc12', 'cfc12|strat_o3', 'ch2cl2', 'ch3ccl3', 'ch3ccl3|strat_o3', 'chcl3', 'co', 'co|trop_o3', 'halon1202', 'halon1202|strat_o3', 'halon1211', 'halon1211|strat_o3', 'halon1301', 'halon1301|strat_o3', 'halon2402', 'halon2402|strat_o3', 'hcfc141b', 'hcfc141b|strat_o3', 'hcfc142b', 'hcfc142b|strat_o3', 'hcfc22', 'hcfc22|strat_o3', 'hfc125', 'hfc134a', 'hfc143a', 'hfc152a', 'hfc227ea', 'hfc23', 'hfc236fa', 'hfc245fa', 'hfc32', 'hfc365mfc', 'hfc4310mee', 'methane', 'methane|strat_h2o', 'methane|trop_o3', 'methyl_bromide', 'methyl_bromide|strat_o3', 'methyl_chloride', 'methyl_chloride|strat_o3', 'nf3',

100%|██████████| 765/765 [00:00<00:00, 1375.24 timestep/s]


In [15]:
metric_results = pd.DataFrame(index=baseline_emms.columns.levels[1],columns=pd.MultiIndex.from_product([['AGWP','AGTP','GWP','GTP'],[5,10,20,50,100,500],['direct','indirect']]))

In [16]:
for gas in baseline_emms.columns.levels[1]:
    RF_diff_dir = (experiment_run['RF'][gas]['default'][gas] - experiment_run['RF']['baseline']['default'][gas]).cumsum()
    RF_diff_indir = (experiment_run['RF'][gas]['default']['Total'] - experiment_run['RF']['baseline']['default']['Total']).cumsum()
    T_diff = (experiment_run['T'][gas]['default'] - experiment_run['T']['baseline']['default'])
    for horizon in [5,10,20,50,100,500]:
        metric_results.loc[gas,('AGWP',horizon,'direct')] = RF_diff_dir.loc[2015+horizon]
        metric_results.loc[gas,('AGWP',horizon,'indirect')] = RF_diff_indir.loc[2015+horizon]
        metric_results.loc[gas,('AGTP',horizon,'direct')] = T_diff.loc[2015+horizon]
        metric_results.loc[gas,('AGTP',horizon,'indirect')] = T_diff.loc[2015+horizon]
        
for gas in baseline_emms.columns.levels[1]:
    for horizon in [5,10,20,50,100,500]:
        metric_results.loc[gas,('GWP',horizon,'direct')] = metric_results.loc[gas,('AGWP',horizon,'direct')]/metric_results.loc['carbon_dioxide',('AGWP',horizon,'direct')]
        metric_results.loc[gas,('GWP',horizon,'indirect')] = metric_results.loc[gas,('AGWP',horizon,'indirect')]/metric_results.loc['carbon_dioxide',('AGWP',horizon,'direct')]
        metric_results.loc[gas,('GTP',horizon,'direct')] = metric_results.loc[gas,('AGTP',horizon,'direct')]/metric_results.loc['carbon_dioxide',('AGTP',horizon,'direct')]
        metric_results.loc[gas,('GTP',horizon,'indirect')] = metric_results.loc[gas,('AGTP',horizon,'direct')]/metric_results.loc['carbon_dioxide',('AGTP',horizon,'indirect')]

In [30]:
def round_to_sf(x,sf):
    if x==0:
        return 0
    if np.isnan(x):
        return '-'
    else:
        num= round(x, sf - int(np.floor(np.log10(abs(x)))))
        if num>10**sf:
            return str(int(num))
        else:
            return str(num)
    
metric_results['GWP'].applymap(lambda x: round_to_sf(x,2)).sort_index()#.to_latex('/home/leachl/Documents/Simple_models/FaIR_v2-0_paper/tmp_tex_tables/default_GWP_values')

5                 10                20             \
                       direct  indirect  direct  indirect  direct  indirect   
Gas name                                                                      
bc                       4510      4650    2610      2710    1490      1550   
c2f6                     7890      7900    8680      8700    9640      9610   
c3f8                     6480      6490    7120      7140    7900      7870   
c4f10                    6580      6590    7240      7250    8020      7990   
c5f12                    6200      6210    6810      6830    7560      7530   
c6f14                    5660      5670    6230      6240    6910      6870   
c7f16                    5610      5620    6170      6180    6840      6800   
c8f18                    5460      5470    6010      6030    6660      6700   
c_c4f8                   6960      6970    7660      7670    8490      8460   
carbon_dioxide            1.0     0.999     1.0     0.997     1.0     0.994   
carbon_tetrachloride     4520      2530    4620      2580    4450      2500   
cf4                      4450      4460    4900      4920    5440      5480   
cfc11                    7820      6230    8210      6550    8330      6590   
cfc113                   6770      6240    7260      6690    7650      7000   
cfc114                   7790      7610    8460      8280    9150      8910   
cfc115                   5610      5580    6150      6120    6760      6680   
cfc12                   11200     10500   12100     11200   12800     11800   
ch2cl2                    129       129    74.2      74.7    42.2      42.9   
ch3ccl3                  1380    -258.0    1050    -197.0     670    -126.0   
chcl3                     233       233     134       135    76.3      77.6   
co                          0      16.9       0       9.8       0      5.63   
halon1202                2260  -13500.0    1450   -8690.0     835   -5060.0   
halon1211                6460  -21700.0    6160  -20700.0    5260  -17800.0   
halon1301                8450  -10100.0    8990  -10700.0    9330  -11200.0   
halon2402                4720  -18900.0    4770  -19200.0    4500  -18200.0   
hcfc141b                 4500      3800    3950      3340    2960      2520   
hcfc142b                 7090      6870    6860      6660    6010      5860   
hcfc22                   8460      8200    7730      7500    6160      5940   
hfc125                   7620      7630    7750      7770    7390      7370   
hfc134a                  5640      5650    5290      5300    4380      4420   
hfc143a                  7860      7870    8250      8270    8350      8320   
hfc152a                  1850      1860    1100      1110     629       639   
hfc227ea                 6170      6180    6360      6380    6210      6250   
hfc23                   11100     11100   12000     12000   13100     13000   
hfc236fa                 6790      6800    7380      7400    8010      7980   
hfc245fa                 5610      5620    4740      4760    3400      3430   
hfc32                    5760      5770    4460      4480    2900      2930   
hfc365mfc                4820      4830    4180      4190    3090      3120   
hfc4310mee               5290      5300    5090      5110    4400      4440   
methane                  95.0       135    85.3       122    66.1      95.3   
methyl_bromide           26.7  -13200.0    15.4   -7630.0    8.75   -4380.0   
methyl_chloride          56.3    -595.0    32.6    -345.0    18.5    -198.0   
nf3                     12200     12200   13400     13300   14700     14600   
nh3                         0         0       0         0       0         0   
nitrous_oxide             286       286     307       308     326       328   
nmvoc                       0      36.5       0      21.2       0      12.2   
nox                         0       138       0      80.1       0      46.0   
nox_avi                     0      1620       0       940       0       539 

In [42]:
get_gas_parameter_defaults().T.applymap(lambda x: round_to_sf(x,3)).sort_index().loc['default'].replace('1e-15','-').to_latex('/home/leachl/Documents/Simple_models/FaIR_v2-0_paper/tmp_tex_tables/default_parameter')